## WFP-01-02-01 Sentinel-2 reflectances and vegetation indices

### <a name="quicklink">Quick link

* [Objective](#objective)
* [Data](#data)
* [Service Definition](#service)
* [Parameter Definition](#parameter)
* [Runtime Parameter Definition](#runtime)
* [Workflow](#workflow)
* [License](#license)

### <a name="objective">Objective 

As a land cover change analyst I want to derive the vegetation indices from Sentinel-2 products.

Co-located timeseries of:

* Sentinel-2 Level 2A reflectances
* Indices NDVI, NDWI, MNDWI, NDBI

### <a name="data">Data 

SENTINEL data products are made available systematically and free of charge to all data users including the general public, scientific and commercial users. Radar data will be delivered within an hour of reception for Near Real-Time (NRT) emergency response, within three hours for NRT priority areas and within 24 hours for systematically archived data.

The data used are Sentinel-2 Level-1C products: Top of atmosphere reflectances in fixed cartographic geometry (combined UTM projection and WGS84 ellipsoid). Level-1C images are a set of tiles of 100 sq km, each of which is approximately 500 MB. These products contain applied radiometric and geometric corrections (including orthorectification and spatial registration). 

The spectral bands of Sentinel-2 Level-1C products are:

| S-2 band                | 1   | 2   | 3   | 4   | 5   | 6   | 7   | 8   | 8a  | 9   | 10   | 11   | 12   |
|-------------------------|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|------|------|------|
| Central wavelength (nm) | 443 | 490 | 560 | 665 | 705 | 740 | 783 | 842 | 865 | 945 | 1375 | 1610 | 2190 |
| Bandwidth (nm)          | 20  | 65  | 35  | 30  | 15  | 15  | 20  | 115 | 20  | 20  | 30   | 90   | 180  |
| Spatial resolution (m)  | 60  | 10  | 10  | 10  | 20  | 20  | 20  | 10  | 20  | 60  | 60   | 20   | 20   |

### <a name="service">Service Definition

In [1]:
service = dict([('title', 'wfp-01-02-01'),
                ('abstract', 'WFP-01-02-01 Sentinel-2 reflectances and vegetation indices'),
                ('id', 'ewf-wfp-01-02-01')])

### <a name="parameter">Parameter Definition 

**Resolution**

Target resolution expressed in meters: 10, 20 or 60

In [2]:
resolution = dict([('id', 'resolution'),
                   ('value', '10'),
                   ('title', 'Target resolution'),
                   ('abstract', 'Target resolution (10m, 20m or 60m)')])

**Area of interest WKT**

Area of interest expressed in Well-Known Text

In [3]:
wkt = dict([('id', 'wkt'),
            ('value', 'POLYGON((14.9997721265064 38.0212298555682,15.0339421243074 38.1469295398089,15.076722011609 38.2943830685215,15.1196939590664 38.4418167678655,15.1628416993738 38.5892613166416,15.2061510925436 38.7366798078584,15.2388261265461 38.8477042183055,16.2649601148006 38.8421486728812,16.2478654087227 37.8528295032852,14.9997726359092 37.8594419593273,14.9997721265064 38.0212298555682))'),
            ('title', 'Protected Area wkt'),
            ('abstract', 'Protected Area wkt')])


** Sentinel-2 flag expression**

Flag expression for pixel selection

In [4]:
flag_expr = dict([('id', 'flag_expr'),
                  ('value', '( saturated_l1a_B4 or scl_water )'),
                  ('title', 'Flag expression for pixel exclusion'),
                  ('abstract', 'Flag expression for pixel exclusion (e.g. saturated_l1a_B4 will exclude pixels having the flag saturated_l1a_B4 set)')])

### <a name="runtime">Runtime parameter definition

**Input identifier**

This is the Sentinel-2 product identifier

In [5]:
input_identifier = 'S2B_MSIL2A_20180803T094029_N0208_R036_T33SWC_20180803T141742'

**Input reference**

This is the Sentinel-2 catalogue reference

In [6]:
input_reference = 'https://catalog.terradue.com/sentinel2/search?uid=S2B_MSIL2A_20180803T094029_N0208_R036_T33SWC_20180803T141742'

**Data path**

This path defines where the data is staged-in. 

In [7]:
data_path = '/data2'

### <a name="workflow">Workflow

#### Import the packages required for processing the Sentinel-1 backscatter

In [8]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
import os
import sys

import dateutil.parser as parser

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import snappy

import gc

import gdalnumeric

import gdal
import osr
import numpy as np

import cioppy
import lxml.etree as etree

from shapely.wkt import loads

import gdal
import osr

from shapely.geometry import box

import gzip
import shutil


#### Open the Sentinel-2 product

In [9]:
s2prd = "%s/%s/%s.SAFE/MTD_MSIL2A.xml" % (data_path, input_identifier, input_identifier)
product = snappy.ProductIO.readProduct(s2prd)

width = product.getSceneRasterWidth()
height = product.getSceneRasterHeight()
name = product.getName()
description = product.getDescription()
band_names = product.getBandNames()



In [10]:
product_date = parser.parse(product.getStartTime().toString()).date()
output_date = '%s%02d%02d' % (product_date.year, product_date.month, product_date.day)


#### Resample 

In [11]:
if resolution['value'] == '10':
    reference_band = 'B4'

if resolution['value'] == '20':
    reference_band = 'B5'
    
if resolution['value'] == '60':
    reference_band = 'B1' 

In [12]:
snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

HashMap = snappy.jpy.get_type('java.util.HashMap')
   
parameters = HashMap()
parameters.put('referenceBand', reference_band)

resample = snappy.GPF.createProduct('Resample', parameters, product)

In [13]:
# NDVI, NDWI, MNDWI, NDBI

ndvi_expr = '((B8 - B4) / (B8 + B4)) * 10000'
ndwi_expr = '((B3 - B8) / ( B3 + B8)) * 10000'
mndwi_expr = '((B3 - B11) / (B3 + B11 )) * 10000'
ndbi_expr = '((B11 - B8) / (B11 + B8 )) * 10000'

if flag_expr['value']:

    ndvi_expr = '! %s ? %s : -20000' % (flag_expr['value'], ndvi_expr)
    ndwi_expr = '! %s ? %s : -20000' % (flag_expr['value'], ndwi_expr)
    mndwi_expr = '! %s ? %s : -20000' % (flag_expr['value'], mndwi_expr)
    ndbi_expr = '! %s ? %s : -20000' % (flag_expr['value'], ndbi_expr) 

In [17]:
band_names = resample.getBandNames()
list(band_names)[0:12]

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'quality_aot',
 'quality_wvp',
 'quality_cloud_confidence',
 'quality_snow_confidence',
 'quality_scene_classification',
 'view_zenith_mean',
 'view_azimuth_mean',
 'sun_zenith',
 'sun_azimuth',
 'view_zenith_B1',
 'view_azimuth_B1',
 'view_zenith_B2',
 'view_azimuth_B2',
 'view_zenith_B3',
 'view_azimuth_B3',
 'view_zenith_B4',
 'view_azimuth_B4',
 'view_zenith_B5',
 'view_azimuth_B5',
 'view_zenith_B6',
 'view_azimuth_B6',
 'view_zenith_B7',
 'view_azimuth_B7',
 'view_zenith_B8',
 'view_azimuth_B8',
 'view_zenith_B8A',
 'view_azimuth_B8A',
 'view_zenith_B9']

In [18]:
target_bands = list(band_names)[0:12] + ['ndvi', 
                                         'ndwi',
                                         'mndwi',
                                         'ndbi',
                                         'quality_cloud_confidence',
                                         'quality_snow_confidence',
                                         'quality_scene_classification',
                                         'opaque_clouds_10m']

In [19]:
BandDescriptor = snappy.jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')

targetBands = snappy.jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', len(target_bands))

for index, band in enumerate(target_bands):
     
    targetBand = BandDescriptor()
    
    if band == 'ndvi':
       
        targetBand.expression = ndvi_expr 
    
    elif band == 'ndwi':
        
        targetBand.expression = ndwi_expr 
    
    elif band == 'mndwi':
        
        targetBand.expression = mndwi_expr 

    elif band == 'ndbi':
    
         targetBand.expression = ndbi_expr 
    
    else:
    
        targetBand.expression = band

    targetBand.name = band
    targetBand.type = 'float32'
    
    targetBands[index]= targetBand
        

parameters = HashMap()
parameters.put('targetBands', targetBands)

result = snappy.GPF.createProduct('BandMaths', parameters, resample)

#### Subset

In [20]:
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')

geom = WKTReader().read(wkt['value'])

parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('geoRegion', geom)
    
subset_result = snappy.GPF.createProduct('Subset', parameters, result)


In [22]:
output_name = '%s_VI_BOA' % input_identifier

snappy.ProductIO.writeProduct(subset_result, output_name, 'GeoTIFF-BigTIFF')

## Identify cloud cover over the AOI

In [28]:
ds = gdal.Open(output_name+ '.tif')
raster_file = np.array(ds.GetRasterBand(20).ReadAsArray())


print raster_file.min(), raster_file.max()
pixel_count_cloud_geo = (raster_file == 255).sum()  # for pixel value = 1

cloud_percent =  float(pixel_count_cloud_geo) / float(raster_file.size) * 100.0
print cloud_percent

raster_file = None

0.0 255.0
4.1278927409


### Metadata

In [36]:
def eop_metadata(metadata):

    opt = 'http://www.opengis.net/opt/2.1'
    om  = 'http://www.opengis.net/om/2.0'
    gml = 'http://www.opengis.net/gml/3.2'
    eop = 'http://www.opengis.net/eop/2.1'
    sar = 'http://www.opengis.net/sar/2.1'
    
    root = etree.Element('{%s}EarthObservation' % opt)

    phenomenon_time = etree.SubElement(root, '{%s}phenomenonTime' % om)

    time_period = etree.SubElement(phenomenon_time, '{%s}TimePeriod' % gml)

    begin_position = etree.SubElement(time_period, '{%s}beginPosition'  % gml)

    end_position = etree.SubElement(time_period, '{%s}endPosition'  % gml)

    procedure = etree.SubElement(root, '{%s}procedure' % om)

    earth_observation_equipment = etree.SubElement(procedure, '{%s}EarthObservationEquipment' % eop)

    acquisition = etree.SubElement(earth_observation_equipment, '{%s}acquisitionParameters' % eop)

    orbit_number = etree.SubElement(acquisition, '{%s}orbitNumber' % eop)

    wrs_longitude_grid = etree.SubElement(acquisition, '{%s}wrsLongitudeGrid' % eop)
 
    feature_of_interest = etree.SubElement(root, '{%s}featureOfInterest' % om)
    footprint = etree.SubElement(feature_of_interest, '{%s}Footprint' % eop)
    multi_extentOf = etree.SubElement(footprint, '{%s}multiExtentOf' % eop)
    multi_surface = etree.SubElement(multi_extentOf, '{%s}MultiSurface' % gml)
    surface_members = etree.SubElement(multi_surface, '{%s}surfaceMembers' % gml)
    polygon = etree.SubElement(surface_members, '{%s}Polygon' % gml)    
    exterior = etree.SubElement(polygon, '{%s}exterior' % gml)  
    linear_ring = etree.SubElement(exterior, '{%s}LinearRing' % gml) 
    poslist = etree.SubElement(linear_ring, '{%s}posList' % gml) 


    result = etree.SubElement(root, '{%s}result' % om)
    earth_observation_result = etree.SubElement(result, '{%s}EarthObservationResult' % opt)
    cloud_cover_percentage = etree.SubElement(earth_observation_result, '{%s}cloudCoverPercentage' % opt)
    
    metadata_property = etree.SubElement(root, '{%s}metaDataProperty' % eop)
    earth_observation_metadata = etree.SubElement(metadata_property, '{%s}EarthObservationMetaData' % eop)
    identifier = etree.SubElement(earth_observation_metadata, '{%s}identifier' % eop)
    
    begin_position.text = metadata['startdate']
    end_position.text = metadata['enddate']
    orbit_number.text = metadata['orbitNumber']
    
    coords = np.asarray([t[::-1] for t in list(loads(metadata['wkt']).exterior.coords)]).tolist()
 
    pos_list = ''
    for elem in coords:
        pos_list += ' '.join(str(e) for e in elem) + ' '   

    poslist.attrib['count'] = str(len(coords))
    poslist.text = pos_list
    
    cloud_cover_percentage.text = metadata['cc']
    
    identifier.text = metadata['identifier'] 

    return etree.tostring(root, pretty_print=True)

#### Get the result WKT


In [37]:
src = gdal.Open(output_name + '.tif')
ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()

max_x = ulx + (src.RasterXSize * xres)
min_y = uly + (src.RasterYSize * yres)
min_x = ulx 
max_y = uly

source = osr.SpatialReference()
source.ImportFromWkt(src.GetProjection())

target = osr.SpatialReference()
target.ImportFromEPSG(4326)

transform = osr.CoordinateTransformation(source, target)

result_wkt = box(transform.TransformPoint(min_x, min_y)[0],
        transform.TransformPoint(min_x, min_y)[1],
        transform.TransformPoint(max_x, max_y)[0],
        transform.TransformPoint(max_x, max_y)[1]).wkt

#### Create the EOP XML metadata file

In [39]:
ciop = cioppy.Cioppy()
search = ciop.search(end_point=input_reference,
                     params=[],
                     output_fields='enclosure,identifier,startdate,enddate,wkt,orbitNumber',
                     model='EOP')      

# update the content of the metadata with the result values
search[0]['identifier'] = output_name
search[0]['wkt'] = result_wkt
search[0]['cc'] = str(cloud_percent)

eop_xml = output_name + '.xml'
with open(eop_xml, 'wb') as file:
    file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    file.write(eop_metadata(search[0]))

#### Create the properties file for the reproducibility notebooks

In [40]:
for properties_file in ['result', 'stage-in']:

    if properties_file == 'result':
        title = 'Reproducibility notebook used for generating %s' % output_name
    else: 
        title = 'Reproducibility stage-in notebook for Sentinel-2 data for generating %s' % output_name
        
    with open(properties_file + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (search[0]['startdate'], search[0]['enddate']))
        file.write('geometry=%s' % (search[0]['wkt']))
        

### Compress the file

In [41]:
with open(output_name + '.tif', 'rb') as f_in, gzip.open(output_name + '.gz', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)
    
os.remove(output_name + '.tif')

OSError: [Errno 2] No such file or directory: 'S2B_MSIL2A_20180803T094029_N0208_R036_T33SWC_20180803T141742_VI_BOA.tif'

### <a name="license">License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.